In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, count, to_date, col, current_timestamp, round


StatementMeta(, 8356fa6e-8601-4db7-bba4-52d29aa3b3e3, 3, Finished, Available, Finished)

In [2]:
# 1️⃣ Read from base Gold Fact_Orders
fact_orders = spark.read.format("delta").table("GoldLakehouse.dbo.fact_orders")

# 2️⃣ Create daily sales summary
agg_sales_summary = (
    fact_orders
    .withColumn("Order_Date", to_date(col("Order_Date")))
    .groupBy("Order_Date", "Product_Category", "Product_Brand")
    .agg(
        round(sum("Order_Total"), 2).alias("Total_Sales"),
        round(avg("Price"), 2).alias("Average_Price"),
        count("Order_ID").alias("Total_Orders"),
        sum("Quantity").alias("Total_Quantity")
    )
)


StatementMeta(, 8356fa6e-8601-4db7-bba4-52d29aa3b3e3, 4, Finished, Available, Finished)

In [3]:
# 3️⃣ Add gold ingestion timestamp
agg_sales_summary = agg_sales_summary.withColumn("gold_ingestion_time", current_timestamp())

# 4️⃣ Write to Gold as Aggregated Table
agg_sales_summary.write.format("delta").mode("overwrite").saveAsTable("Gold_Agg_Sales_Summary")



StatementMeta(, 8356fa6e-8601-4db7-bba4-52d29aa3b3e3, 5, Finished, Available, Finished)